In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,precision_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [7]:
data=pd.read_csv("bugzilla.csv")
data_accross_project=pd.read_csv("columba.csv")

In [16]:
data_accross_project.head(5)
data_ap=data_accross_project
data_ap=data_ap.drop('transactionid', axis=1)
data_ap=data_ap.drop('commitdate', axis=1)
data_ap.head()
X_data_ap=data_ap.iloc[:,:-1]
Y_data_ap=data_ap.iloc[:,-1]

In [9]:
data.head()

,transactionid,commitdate,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp,bug
0,3,2001/12/12 17:41,1,1,3,0.579380,0.093620,0.000000,480.666667,1,14,596,0.666667,143,133.50,129,1
1,7,1999/10/12 12:57,1,1,1,0.000000,0.000000,0.000000,398.000000,1,1,0,1.000000,140,140.00,137,1
2,8,2002/5/15 16:55,3,3,52,0.739279,0.183477,0.208913,283.519231,0,23,15836,0.750000,984,818.65,978,0
3,9,2002/1/21 15:37,1,1,8,0.685328,0.016039,0.012880,514.375000,1,21,1281,1.000000,579,479.25,550,0
4,10,2001/12/19 16:44,2,2,38,0.769776,0.091829,0.072746,366.815789,1,21,6565,0.763158,413,313.25,405,0


In [10]:
data = data.sample(frac=1)

# total bug data classes 1696 rows.
non_bug_data = data.loc[data['bug'] == 0][:1000]
bug = data.loc[data['bug'] == 1][:1000]
test_data_1=data.loc[data['bug'] == 1][1000:1696]
test_data_0 = data.loc[data['bug'] == 0][1000:-1]
pure_data = pd.concat([non_bug_data,bug])

# Shuffle dataframe rows
pure_data = pure_data.sample(frac=1, random_state=7)
pure_data=pure_data.drop("transactionid", axis=1)
pure_data=pure_data.drop("commitdate", axis=1)

X=pure_data.iloc[:,:-1]
Y=pure_data.iloc[:,-1]

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=20)


In [25]:
def deep_enesmble(X_train,Y_train):
    l=[]
    X_train1,X_test1,Y_train1,Y_test1=train_test_split(X_train,Y_train,test_size=0.2,random_state=20)
    model=Sequential()
    model.add(Dense(128, init="uniform", input_dim=14, activation='relu'))
    model.add(Dense(64, init ="uniform", activation="relu"))
    model.add(Dense(1, init="uniform", activation="sigmoid"))
    model.compile(loss="binary_crossentropy", metrics=['accuracy'], optimizer='adam')
    history=model.fit(X_train1,Y_train1, epochs=100, batch_size=100)
    Y_pred1=model.predict(X_test1)
    Y_pred1=(Y_pred1>0.5)
    model3=RandomForestClassifier()
    model3.fit(X_train,Y_train)
    Y_pred3=model3.predict(X_test1)
    model2 = XGBClassifier()
    model2.fit(X_train1, Y_train1)
    Y_pred_xg=model2.predict(X_test1)
    accuracy_1=accuracy_score(Y_test1,Y_pred_xg)
    accuracy_2=accuracy_score(Y_test1,Y_pred1)
    accuracy_3=accuracy_score(Y_test1,Y_pred3)
    p1=precision_score(Y_test1,Y_pred1)
    p2=precision_score(Y_test1,Y_pred_xg)
    p3=precision_score(Y_test1,Y_pred3)
    
    f1=f1_score(Y_test1,Y_pred1)
    f2=f1_score(Y_test1,Y_pred_xg)
    f3=f1_score(Y_test1,Y_pred3)
    
    cm=confusion_matrix(Y_test1.tolist(),Y_pred1.tolist())
    cm2=confusion_matrix(Y_test1.tolist(),Y_pred_xg.tolist())
    cm3=confusion_matrix(Y_test1.tolist(),Y_pred3.tolist())
    
    total=sum(sum(cm))
    sensitivity1 = cm[0,0]/(cm[0,0]+cm[1,0])
    total2=sum(sum(cm2))
    sensitivity2 = cm2[0,0]/(cm2[0,0]+cm2[1,0])
    sensitivity3 = cm3[0,0]/(cm3[0,0]+cm3[1,0])
    
    specificity1 = cm[1,1]/(cm[1,1]+cm[0,1])
    specificity2 = cm2[1,1]/(cm2[1,1]+cm2[0,1])
    specificity3 = cm3[1,1]/(cm3[1,1]+cm3[0,1])
    
    total=sum(sum(cm3))

    return ((accuracy_1+accuracy_2+accuracy_3)/3),(p1+p2+p3)/3, (f1+f2+f3)/3, (specificity1+sensitivity2+sensitivity3)/3, (specificity1+specificity2+specificity3)/3
    

In [26]:
a,b,c,d,e=deep_enesmble(X_train,Y_train)

/home/rosa-mystica/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=14, activation="relu", kernel_initializer="uniform")`
  """
/home/rosa-mystica/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
  
/home/rosa-mystica/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


Epoch 1/100
1280/1280 [==============================] - 0s 191us/step - loss: 0.7049 - acc: 0.5430
Epoch 2/100
1280/1280 [==============================] - 0s 16us/step - loss: 0.6987 - acc: 0.5742
Epoch 3/100
1280/1280 [==============================] - 0s 23us/step - loss: 0.6721 - acc: 0.5828
Epoch 4/100
1280/1280 [==============================] - 0s 18us/step - loss: 0.6485 - acc: 0.6039
Epoch 5/100
1280/1280 [==============================] - 0s 20us/step - loss: 0.6420 - acc: 0.6031
Epoch 6/100
1280/1280 [==============================] - 0s 20us/step - loss: 0.6455 - acc: 0.6227
Epoch 7/100
1280/1280 [==============================] - 0s 19us/step - loss: 0.6489 - acc: 0.6047
Epoch 8/100
1280/1280 [==============================] - 0s 20us/step - loss: 0.6469 - acc: 0.6195
Epoch 9/100
1280/1280 [==============================] - 0s 21us/step - loss: 0.6413 - acc: 0.6258
Epoch 10/100
1280/1280 [==============================] - 0s 19us/step - loss: 0.6441 - acc: 0.6102
Epoch 11

1280/1280 [==============================] - 0s 23us/step - loss: 0.5296 - acc: 0.7109
Epoch 84/100
1280/1280 [==============================] - 0s 20us/step - loss: 0.5199 - acc: 0.7141
Epoch 85/100
1280/1280 [==============================] - 0s 20us/step - loss: 0.5150 - acc: 0.7141
Epoch 86/100
1280/1280 [==============================] - 0s 24us/step - loss: 0.5187 - acc: 0.7070
Epoch 87/100
1280/1280 [==============================] - 0s 23us/step - loss: 0.5129 - acc: 0.7234
Epoch 88/100
1280/1280 [==============================] - 0s 22us/step - loss: 0.5169 - acc: 0.7187
Epoch 89/100
1280/1280 [==============================] - 0s 21us/step - loss: 0.5191 - acc: 0.7086
Epoch 90/100
1280/1280 [==============================] - 0s 22us/step - loss: 0.5071 - acc: 0.7180
Epoch 91/100
1280/1280 [==============================] - 0s 20us/step - loss: 0.5129 - acc: 0.7172
Epoch 92/100
1280/1280 [==============================] - 0s 21us/step - loss: 0.5224 - acc: 0.7227
Epoch 93/100


/home/rosa-mystica/anaconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [27]:
a,b,c,d,e

(0.7739583333333333,
 0.7842245035826219,
 0.7546382257907908,
 0.7798321506108032,
 0.7842245035826219)

In [44]:
def tree(pure_data, n):
    j=0
    accuracy=0
    p=0
    f1_score=0
    sensitivity=0
    specificity=0
    for i in range(n):
        x1=pure_data.iloc[j:j+50,:-1]
        y1=pure_data.iloc[j:j+50,-1]
        j=j+50
        a,p1,f,sens,spec=deep_enesmble(x1,y1)
        accuracy=a+accuracy
        p=p+p1
        f1_score=f1_score+f
        sensitivity=sens+sensitivity
        specificity=specificity+spec
    print("Accuracy",accuracy)
    print("P score", p)
    print("f1_score",f1_score)
    print("Sensivity",sensitivity)
    print("specificity",specificity)
        
        
        


    

In [ ]:
"""CHECKING FOR THE PERFORMANCE USING DIFFERENT WHOLE DATASET"""

In [28]:
a2,b2,c2,d2,e2=deep_enesmble(X_data_ap,Y_data_ap)

/home/rosa-mystica/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=14, activation="relu", kernel_initializer="uniform")`
  """
/home/rosa-mystica/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
  
/home/rosa-mystica/anaconda3/envs/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


Epoch 1/100
3564/3564 [==============================] - 0s 83us/step - loss: 1.3794 - acc: 0.6164
Epoch 2/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.7673 - acc: 0.6411
Epoch 3/100
3564/3564 [==============================] - 0s 12us/step - loss: 0.6622 - acc: 0.6911
Epoch 4/100
3564/3564 [==============================] - 0s 14us/step - loss: 0.7199 - acc: 0.6692
Epoch 5/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.7215 - acc: 0.6580
Epoch 6/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.6538 - acc: 0.6930
Epoch 7/100
3564/3564 [==============================] - 0s 12us/step - loss: 0.6681 - acc: 0.6860
Epoch 8/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.6582 - acc: 0.6846
Epoch 9/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.6263 - acc: 0.7079
Epoch 10/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.6256 - acc: 0.7059
Epoch 11/

3564/3564 [==============================] - 0s 14us/step - loss: 0.5558 - acc: 0.7346
Epoch 84/100
3564/3564 [==============================] - 0s 12us/step - loss: 0.5480 - acc: 0.7332
Epoch 85/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.5565 - acc: 0.7301
Epoch 86/100
3564/3564 [==============================] - 0s 12us/step - loss: 0.5437 - acc: 0.7379
Epoch 87/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.5428 - acc: 0.7416
Epoch 88/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.5406 - acc: 0.7385
Epoch 89/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.5779 - acc: 0.7256
Epoch 90/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.5534 - acc: 0.7368
Epoch 91/100
3564/3564 [==============================] - 0s 13us/step - loss: 0.5458 - acc: 0.7374
Epoch 92/100
3564/3564 [==============================] - 0s 12us/step - loss: 0.5485 - acc: 0.7407
Epoch 93/100


/home/rosa-mystica/anaconda3/envs/myenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [29]:
a2,b2,c2,d2,e2

(0.8185559296670407,
 0.7993556892453951,
 0.5860345999347204,
 0.8162019780533932,
 0.7993556892453951)